In [ ]:
!git clone https://github.com/Auto-annotation-of-Pathology-Images/AAPI_code.git

This demo is working for google colab

In [ ]:
from AAPI_code.Collage_generator.collage_generator import collage_generator
from AAPI_code.Collage_generator.utils import *

# Importing cropped images

In [ ]:
image_list, label_list = import_git_images()

# Initialization:  
example_image.jpg should be okay for background setting  
you can define another `keras.preprocessing.image.ImageDataGenerator `, when it's None it will use the original setting.

In [ ]:
col_gen = collage_generator(label_list= label_list, 
                            canvas_size=(1024,1024),
                            example_image = "/content/AAPI_code/Collage_generator/example_background_dark.png",
                            gaussian_noise_constant = 100,
                            patience = 20,
                            scanning_constant = 50,
                            imagedatagenerator = None)

# Add Image
`img` can be path, np.ndarray, or PIL image object.  
Personally suggest that set `pre_determined_size = False` first, it can help setting a correct "original size"

In [ ]:
# The final size is not decided, this value is for demo
size_dict = {'glomerulus': (400,400), 'tubules':(250,250), 'artery':(400,400), 'arteriole':(300,300)}

for label,images in zip(label_list, image_list):
    for image in images:
        col_gen.add_image(img = image, 
                          label = label, 
                          size = size_dict[label], 
                          pre_determined_size = True)

# Generate

Just like that... `ratio_list` defines the ratio of different entity (glomerulus, artery, etc.) you can call `col_gen.label_list` for the actual order, it's okay to have them not sum to 1.

In [ ]:
collage,mask, dictionary = col_gen.generate(ratio_list = [0.8,0.05,0.1,0.05],
                                            background_color = False)
visualize_result(collage, mask, dictionary)

In [ ]:
collage,mask, dictionary = col_gen.generate(ratio_list = [0.8,0.05,0.1,0.05],
                                            background_color = True)
visualize_result(collage, mask, dictionary)

In [ ]:
col_gen.generate_hdf5("try.h5", # this path can have .h5 extension or not
                      length = 2, # the number of image generated
                      vignettes_ratio_list=[0.8,0.05,0.1,0.05],
                      background_color_ratio=1.0 # Maybe some of the training need white background?
                      )

# generating_dataset

If want to start from raw, a generating_dataset class is implemented if not trying to read hdf5. But this one will not be used often

In [ ]:
from AAPI_code.Collage_generator.generating_dataset import generating_dataset
from torch.utils.data import DataLoader

collage_dataset = generating_dataset(collage_generator= col_gen,
                                     dataset_size = 3)

In [ ]:
EPOCHS = 10

data_loader = DataLoader(collage_dataset, batch_size = 3, shuffle = True)

criterion = nn.CrossEntropyLoss(reduction="mean")

unet = UNet(in_channels=3,
            n_classes=5,
            depth = 8,
            wf = 3,
            padding = True,
            batch_norm = True,
            up_mode = "upconv"
            )

optimizer = torch.optim.Adam(params = unet.parameters(),lr = 1e-3)

scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience = 25, verbose = True)

In [ ]:
i = 0
for epochs in tqdm(range(EPOCHS), desc = "Epochs", leave = True):
    for image, mask in tqdm(data_loader, desc = f"Epoch {epochs}", leave = False):
        result = unet(image)
        loss = criterion(result, mask)
        loss.backward()
        optimizer.step()
        l = loss.item()
        print(l)
    scheduler.step(l)